[課題のURL](https://diver.diveintocode.jp/curriculums/1626)

# Sprintディープラーニングフレームワーク2

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

import tensorflow as tf

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
from keras import losses

# 【問題1】公式チュートリアルモデルを分担して実行

https://www.tensorflow.org/tutorials

KerasによるMLの基本,　保存と読み込み  
→学習に時間がかかるので一度学習したデータを保存したい。主に下記の２点を紹介。

1.   

下記のデータ保存をするコールバック関数をfit内の引数に設定する。   

~~~
    tf.keras.callbacks.ModelCheckpoint()
~~~
→エポックごとに保存するなど。  
→1Epoch終了後に「保存されている重みの「Val_loss」＞学習後の「Val_loss」」の場合の時だけ保存するなど

2.   

重みの値、モデルの設定、オプティマイザの設定ができる。  
~~~
    model.save("my_model.h5")
    models.load_model("my_model.h5")
~~~

## 重みを保存

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

import tensorflow as tf
from tensorflow import keras

tf.__version__

'1.15.0'

In [16]:
# データの読み込み

(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

In [20]:
# 短いシーケンシャルモデルを返す関数
def create_model():
    model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation='softmax')
  ])
  
    model.compile(optimizer='adam', 
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  
    return model

In [9]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# チェックポイントコールバックを作る
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                 save_weights_only=True,
                                                 verbose=1)

model = create_model()

model.fit(train_images, train_labels,  epochs = 10, 
          validation_data = (test_images,test_labels),
          callbacks = [cp_callback])  # 訓練にコールバックを渡す


Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 832/1000 [=======================>......] - ETA: 0s - loss: 1.2551 - acc: 0.6514
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 1s 809us/sample - loss: 1.1621 - acc: 0.6770 - val_loss: 0.7061 - val_acc: 0.8000
Epoch 2/10
 928/1000 [==========================>...] - ETA: 0s - loss: 0.4155 - acc: 0.8912
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 291us/sample - loss: 0.4159 - acc: 0.8880 - val_loss: 0.5415 - val_acc: 0.8470
Epoch 3/10
 864/1000 [========================>.....] - ETA: 0s - loss: 0.2857 - acc: 0.9213
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 362us/sample - loss: 0.2820 - acc: 0.9180 - val_loss: 0.4677 - val_acc: 0.8540
Epoch 4/10
 928/1000 [==========================>...] - ETA: 0s - loss: 0.2183 - acc: 0.9440
Epoch 00004: saving model to training_1/cp.ckpt
1000/10

In [10]:
!ls {checkpoint_dir}

checkpoint                  cp.ckpt.data-00001-of-00002
cp.ckpt.data-00000-of-00002 cp.ckpt.index


In [22]:
# 使用例

In [11]:
K.clear_session()
model = create_model()

loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 - 0s - loss: 2.3353 - acc: 0.1000
Untrained model, accuracy: 10.00%


In [12]:
model.load_weights(checkpoint_path)
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 - 0s - loss: 0.4046 - acc: 0.8720
Restored model, accuracy: 87.20%


## モデルを保存

In [14]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

# モデル全体を１つのHDF5ファイルに保存します。
model.save('my_model.h5')

# 重みとオプティマイザを含む全く同じモデルを再作成
new_model = keras.models.load_model('my_model.h5')
new_model.summary()

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 287us/sample - loss: 1.1638 - acc: 0.6720
Epoch 2/5
1000/1000 [==============================] - 0s 189us/sample - loss: 0.4191 - acc: 0.8870
Epoch 3/5
1000/1000 [==============================] - 0s 201us/sample - loss: 0.2766 - acc: 0.9270
Epoch 4/5
1000/1000 [==============================] - 0s 182us/sample - loss: 0.1847 - acc: 0.9590
Epoch 5/5
1000/1000 [==============================] - 0s 188us/sample - loss: 0.1677 - acc: 0.9570
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 512)               401920    
______________________

## Kerasへの書き換え

KerasはTensorFLowに含まれるtf.kerasモジュールを使用してください。  

KerasにはSequentialモデルかFunctional APIかなど書き方に種類がありますが、これは指定しません。  

## 【問題3】Iris（2値分類）をKerasで学習

TensorFlowによるIrisデータセットに対する2値分類をKerasに書き換えてください。

In [19]:
# -*- coding: utf-8 -*
import keras
print(keras.__version__)

2.3.1


In [32]:
from keras.optimizers import Adam

# データセットの読み込み
dataset_path ="Iris.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
df = df[(df["Species"] == "Iris-versicolor")|(df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y=='Iris-versicolor'] = 0
y[y=='Iris-virginica'] = 1
y = y.astype(np.int)[:, np.newaxis]
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

# ハイパーパラメータの設定
learning_rate = 0.001
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

# モデルの作成
K.clear_session()
model = Sequential()

# モデルにレイヤーを積み上げていく
model.add(Dense(n_hidden1, input_dim=n_input))
model.add(Activation('relu'))
model.add(Dense(n_hidden2, input_dim=n_hidden1))
model.add(Activation('relu'))
model.add(Dense(n_classes))

# 訓練プロセスの定義
model.compile(loss="binary_crossentropy",
              optimizer=Adam(lr = learning_rate),
              metrics=['accuracy'])

# 訓練の実行
# (x_train, y_trainはNumpy行列の学習データ)
model.fit(X_train, y_train, epochs=num_epochs, batch_size=batch_size, validation_split=0.2)

# 予測の実行
y_pred = model.predict_classes(X_test)
score = accuracy_score(y_pred, y_test)
print("test:", score)

Train on 64 samples, validate on 16 samples
Epoch 1/10
64/64 [==============================] - 0s 5ms/step - loss: 0.7517 - accuracy: 0.5781 - val_loss: 0.5963 - val_accuracy: 0.5625
Epoch 2/10
64/64 [==============================] - 0s 633us/step - loss: 0.6747 - accuracy: 0.5781 - val_loss: 0.5246 - val_accuracy: 0.7500
Epoch 3/10
64/64 [==============================] - 0s 523us/step - loss: 0.5456 - accuracy: 0.7656 - val_loss: 0.4909 - val_accuracy: 1.0000
Epoch 4/10
64/64 [==============================] - 0s 643us/step - loss: 0.4984 - accuracy: 0.9219 - val_loss: 0.4739 - val_accuracy: 0.9375
Epoch 5/10
64/64 [==============================] - 0s 587us/step - loss: 0.4688 - accuracy: 0.9375 - val_loss: 0.4384 - val_accuracy: 1.0000
Epoch 6/10
64/64 [==============================] - 0s 445us/step - loss: 0.4414 - accuracy: 0.9219 - val_loss: 0.4008 - val_accuracy: 1.0000
Epoch 7/10
64/64 [==============================] - 0s 478us/step - loss: 0.4313 - accuracy: 0.9219 - val_

In [33]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 50)                250       
_________________________________________________________________
activation_1 (Activation)    (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               5100      
_________________________________________________________________
activation_2 (Activation)    (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 5,451
Trainable params: 5,451
Non-trainable params: 0
_________________________________________________________________
None


# 【問題4】Iris（多値分類）をKerasで学習  

TensorFlowによるIrisデータセットに対する3値分類をKerasに書き換えてください。

In [8]:
from keras.optimizers import Adam

def encode_3(data):
    t = np.zeros((data.size, 3))
    for i in range(data.size):
        t[i, data[i]] = 1
    return t

# データセットの読み込み
dataset_path ="Iris.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
#df = df[(df["Species"] == "Iris-versicolor")|(df["Species"] == "Iris-virginica")]
df = df[(df["Species"] == "Iris-versicolor")|(df["Species"] == "Iris-virginica")|(df["Species"] == "Iris-setosa")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y=='Iris-versicolor'] = 0
y[y=='Iris-virginica'] = 1
y[y=='Iris-setosa'] = 2
y = y.astype(np.int)[:, np.newaxis]
y = encode_3(y)
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割

In [9]:
learning_rate = 0.01
batch_size = 10
num_epochs = 10
n_hidden1 = 5
n_hidden2 = 10
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_samples_test = X_test.shape[0]
# n_samples_val = X_val.shape[0]
n_classes = 3

# モデルの作成
K.clear_session()
model = Sequential()

# モデルにレイヤーを積み上げていく
model.add(Dense(n_hidden1, input_dim=n_input))
model.add(Activation('relu'))
model.add(Dense(n_hidden2, input_dim=n_hidden1))
model.add(Activation('relu'))
model.add(Dense(n_classes))
model.add(Activation('softmax'))

# 訓練プロセスの定義
model.compile(loss="categorical_crossentropy",
              optimizer=Adam(lr = 0.02),
              #optimizer="Adam",
              metrics=['accuracy'])

# 訓練の実行
# (x_train, y_trainはNumpy行列の学習データ)
model.fit(X_train, y_train, epochs=num_epochs, batch_size=batch_size, validation_split=0.2)

# 予測の実行
y_pred = model.predict_classes(X_test)
y_test_decode = np.argmax(y_test, axis=1)
score = accuracy_score(y_pred, y_test_decode)
print("test:", score)


Train on 96 samples, validate on 24 samples
Epoch 1/10
96/96 [==============================] - 0s 4ms/step - loss: 1.0383 - accuracy: 0.4688 - val_loss: 1.0014 - val_accuracy: 0.5417
Epoch 2/10
96/96 [==============================] - 0s 291us/step - loss: 0.7518 - accuracy: 0.7292 - val_loss: 0.9206 - val_accuracy: 0.5417
Epoch 3/10
96/96 [==============================] - 0s 261us/step - loss: 0.5665 - accuracy: 0.7396 - val_loss: 0.6741 - val_accuracy: 0.6250
Epoch 4/10
96/96 [==============================] - 0s 264us/step - loss: 0.4377 - accuracy: 0.8646 - val_loss: 0.6170 - val_accuracy: 0.5417
Epoch 5/10
96/96 [==============================] - 0s 260us/step - loss: 0.3960 - accuracy: 0.7500 - val_loss: 0.4345 - val_accuracy: 0.7500
Epoch 6/10
96/96 [==============================] - 0s 326us/step - loss: 0.3568 - accuracy: 0.8125 - val_loss: 0.5241 - val_accuracy: 0.6667
Epoch 7/10
96/96 [==============================] - 0s 283us/step - loss: 0.2945 - accuracy: 0.8958 - val_

In [10]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 5)                 25        
_________________________________________________________________
activation_1 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                60        
_________________________________________________________________
activation_2 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 33        
_________________________________________________________________
activation_3 (Activation)    (None, 3)                 0         
Total params: 118
Trainable params: 118
Non-trainable params: 0
________________________________________________________

# 【問題5】House PricesをKerasで学習

TensorFlowによるHouse Pricesデータセットに対する回帰をKerasに書き換えてください。

In [11]:
df = pd.read_csv('train.csv', index_col=0)

# GrLivAreaとYearBuiltを抜き出す。
X = df[["GrLivArea", "YearBuilt"]]
y = df["SalePrice"]

df = pd.concat([X, y], axis=1)

# 対数変換
df["SalePrice"] = np.log(df["SalePrice"])
df["GrLivArea"] = np.log(df["GrLivArea"])
df["YearBuilt"] = np.log(df["YearBuilt"])

# 外れ値の除去
df.drop(df[(df["GrLivArea"]>4500) & (df['SalePrice']<300000)].index, inplace=True)
df.reset_index(drop=True, inplace=True)

X = df[["GrLivArea", "YearBuilt"]]
#X = df[["GrLivArea"]]
y = df["SalePrice"]
# df = pd.concat([X, y], axis=1)
#df.head(2)

X = np.array(X)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [18]:
from keras.optimizers import Adam
# [X] 
# ハイパーパラメータの設定
lr = 0.001
batch_size = 100
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 10 # 利用しない
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

# モデルの作成
K.clear_session()
model = Sequential()

# モデルにレイヤーを積み上げていく
model.add(Dense(n_hidden1, input_dim=n_input))
model.add(Activation('relu'))
model.add(Dense(n_classes, input_dim=n_hidden1))
model.add(Activation('relu'))

# 訓練プロセスの定義
model.compile(loss="mean_squared_error",
              optimizer=Adam(lr = 0.01))
              #optimizer="Adam",
              #metrics=['mean_squared_error'])

# 訓練の実行
# (x_train, y_trainはNumpy行列の学習データ)
model.fit(X_train, y_train, epochs=num_epochs, batch_size=batch_size, validation_split=0.2)

# 予測の実行
y_pred = model.predict(X_test)
#socre = (np.square(y_pred - y_test)).mean(axis=1)
score = mean_squared_error(y_pred, y_test)
print("test:", score)


Train on 934 samples, validate on 234 samples
Epoch 1/10
934/934 [==============================] - 0s 224us/step - loss: 60.7484 - val_loss: 11.1883
Epoch 2/10
934/934 [==============================] - 0s 40us/step - loss: 3.2472 - val_loss: 5.4544
Epoch 3/10
934/934 [==============================] - 0s 35us/step - loss: 4.5648 - val_loss: 0.5116
Epoch 4/10
934/934 [==============================] - 0s 31us/step - loss: 0.4607 - val_loss: 0.9881
Epoch 5/10
934/934 [==============================] - 0s 38us/step - loss: 0.6258 - val_loss: 0.0574
Epoch 6/10
934/934 [==============================] - 0s 28us/step - loss: 0.1657 - val_loss: 0.1823
Epoch 7/10
934/934 [==============================] - 0s 31us/step - loss: 0.1096 - val_loss: 0.0703
Epoch 8/10
934/934 [==============================] - 0s 31us/step - loss: 0.1007 - val_loss: 0.0581
Epoch 9/10
934/934 [==============================] - 0s 30us/step - loss: 0.0778 - val_loss: 0.0623
Epoch 10/10
934/934 [=====================

# 【問題6】MNISTをKerasで学習

TensorFlowによるMNISTデータセットによる画像の多値分類をKerasに書き換えてください。

In [182]:
from keras.datasets import mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

# print(X_train.shape) # (60000, 28, 28)
# print(X_test.shape) # (10000, 28, 28)

X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255
# print(X_train.max()) # 1.0
# print(X_train.min()) # 0.0
print(X_train.shape)
print(X_test.shape)

(60000, 28, 28)
(10000, 28, 28)


In [200]:
from keras.optimizers import Adam

def _encode_10(data):
    t = np.zeros((data.size, 10))
    for i in range(data.size):
        t[i, data[i]] = 1
    return t

"""
Kerasで実装したニューラルネットワークを使いMINSTを分類する
"""
# ハイパーパラメータの設定
learning_rate = 0.001
batch_size = 20
num_epochs = 10
n_hidden1 = 40
n_hidden2 = 20
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_samples_test = X_test.shape[0]
n_samples_val = X_val.shape[0]
n_classes = 10

filter_num = 30
filter_size = 5
filter_pad = 0
filter_stride = 1
hidden_size = 100
input_size = 28
output_size = 10
max_pool_size1 = 2

# モデルの作成
K.clear_session()
model = Sequential()

model.add(Conv2D(n_hidden1, (filter_size, filter_size), activation='relu'))
model.add(MaxPooling2D(pool_size=(max_pool_size1, max_pool_size1)))
model.add(Flatten())
model.add(Dense(n_hidden2, activation='relu'))
model.add(Dense(output_size, activation='softmax'))

# 訓練プロセスの定義
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr = 0.001),
              metrics=['accuracy'])

# データの型を合わせる
X_train_4d = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
y_train_one_hot = _encode_10(y_train)

X_test_4d = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)
y_test_one_hot = _encode_10(y_test)

model.fit(X_train_4d, y_train_one_hot, epochs=num_epochs, batch_size=batch_size, validation_split=0.2)

# 予測の実行
X_test_4d = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)
y_test_one_hot = _encode_10(y_test)

y_pred = model.predict_classes(X_test_4d)

score = accuracy_score(y_pred, y_test)
print("test:", score)

Train on 48000 samples, validate on 12000 samples
Epoch 1/10
48000/48000 [==============================] - 65s 1ms/step - loss: 0.1929 - accuracy: 0.9416 - val_loss: 0.0698 - val_accuracy: 0.9787
Epoch 2/10
48000/48000 [==============================] - 50s 1ms/step - loss: 0.0646 - accuracy: 0.9803 - val_loss: 0.0690 - val_accuracy: 0.9777
Epoch 3/10
48000/48000 [==============================] - 63s 1ms/step - loss: 0.0453 - accuracy: 0.9861 - val_loss: 0.0672 - val_accuracy: 0.9803
Epoch 4/10
48000/48000 [==============================] - 53s 1ms/step - loss: 0.0348 - accuracy: 0.9889 - val_loss: 0.0506 - val_accuracy: 0.9860
Epoch 5/10
48000/48000 [==============================] - 48s 996us/step - loss: 0.0266 - accuracy: 0.9913 - val_loss: 0.0592 - val_accuracy: 0.9835
Epoch 6/10
48000/48000 [==============================] - 43s 894us/step - loss: 0.0218 - accuracy: 0.9928 - val_loss: 0.0525 - val_accuracy: 0.9853
Epoch 7/10
48000/48000 [==============================] - 60s 1m